In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rc("figure", dpi=100)
import numpy as np
import jack
from lmz import *

In [ ]:
import scanpy as sc

# LOAD DATA 

In [ ]:
'''
zedata = [sc.read('/home/ubuntu/benchdata/'+data) for data in "Immune_ALL_hum_mou.h5ad  Immune_ALL_human.h5ad  Lung_atlas_public.h5ad  human_pancreas_norm_complexBatch.h5ad".split()]
batch,typ = Transpose (Map(lambda x:x.split(), 'batch final_annotation#batch final_annotation#batch cell_type#tech celltype'.split("#")))
zedata =  [[z[z.obs[b]==i] for i in z.obs[b].unique()] for z,b in zip(zedata, batch)]
def samplecopy(data, num):
    obs_indices = np.random.choice(data.n_obs, size=num, replace=True,) 
    r=  data[obs_indices].copy()
    r.obs_names_make_unique()
    return r
data = [[samplecopy(d,500) for d in z] for z in zedata]
'''


# i think this was the way:
zedata = [sc.read('/home/ubuntu/benchdata/'+data) for data in "Immune_ALL_hum_mou.h5ad  Immune_ALL_human.h5ad  Lung_atlas_public.h5ad  human_pancreas_norm_complexBatch.h5ad".split()]
batch,typ = Transpose (Map(lambda x:x.split(), 'batch final_annotation#batch final_annotation#batch cell_type#tech celltype'.split("#")))

zedata =  [[z[z.obs[b]==i] for i in z.obs[b].unique()] for z,b in zip(zedata, batch)]
def samplecopy(data,num, seed):
    np.random.seed(seed) 
    obs_indices = np.random.choice(data.n_obs, size=num, replace=True,) 
    r=  data[obs_indices].copy()
    r.obs_names_make_unique()
    return r

adata2 = [[samplecopy(i,750,1234)  for i in series]  for series in zedata]




In [ ]:
# integrate 
i = -1
data = adata2[i][:4]
batchi = batch[i]
labeli = typ[i]

In [ ]:
from cellsaw import merge as merg
import cellsaw as cs
from cellsaw.merge.diffusion import kernel
from sklearn.decomposition import PCA


cs.preprocess.annotate_genescore2(data,selector = 'cell_ranger',  normrow = True, log = True)
me = merg.Merge(data,umaps=[],oldcut=False, pca = 40)

dist = kernel.linear_assignment_integrate(me.projections[1])
mine = kernel.distmatrixumap(me.projections[1],dist,components = 40)
mine_PCA = PCA().fit_transform(mine) 

mnn = sc.external.pp.mnn_correct(*me.data, n_jobs = 30)
mnn_PCA = PCA().fit_transform(mnn[0].X) 

In [ ]:
for e in me.projections[0]:
    print(e.shape)
    print(type(e))

In [ ]:
from scipy import sparse
from sklearn.decomposition import PCA
stack = np.vstack(me.projections[0])
raw_PCA = PCA(n_components = 2).fit_transform(stack)
raw_PCA.shape

# comparison plots  (also f3 from mnn paper)

In [ ]:
from cellsaw import preprocess as pp 
from cellsaw import merge as merg
from cellsaw import draw 
from cellsaw import util
import umap
from sklearn.decomposition import PCA
import ubergauss.tools as t

def pcaplot(adata,d2stack):
    
    d2split =  [d2stack[adata.obs[batchi]==i].copy() for i in adata.obs[batchi].unique()]
    
    labelstack = [ x  for a in me.data for x in a.obs[labeli] ]
    y =  np.array(t.labelsToIntList(labelstack)[0])
    labels =  [y[adata.obs[batchi]==i].copy() for i in adata.obs[batchi].unique()]
    
    def scatter(i,**other):
        x = d2split[i]
        y = labels[i]
        plt.scatter(x[:,0],x[:,1],c=y,**other)
    scatter(0,marker='o', alpha = .3)
    scatter(1,marker="o", alpha = .9)
    scatter(2,marker="x", alpha = .3)
    plt.show()


pcaplot(mnn[0],raw_PCA)
pcaplot(mnn[0],mnn_PCA)
pcaplot(mnn[0],mine_PCA)